In [1]:
import numpy as np
import os
from scipy.stats import norm
import math
import scipy.optimize as opt
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from pandas.tseries.offsets import DateOffset
from dateutil.relativedelta import relativedelta
from scipy.optimize import fsolve, root
import datetime
import matplotlib.ticker as mtick 
import datetime
import statsmodels.api as sm

In [6]:
import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

In [7]:
# latex_width = 6.69423 inches
w = 6.69423 * 9/10
golden_ratio = (5**.5 - 1) / 2
h = w * golden_ratio
h

3.7235355016582883

In [23]:
maturities = np.array([0, 1,2,3,5,7,10,12,15,20,25])
swap_rates = np.array([0.042, 0.042, 0.043, 0.047, 0.054, 0.057, 0.06, 0.061, 0.059, 0.056, 0.0555])

maturities_2 = np.array([0,	0.99,1,1.99,2,2.99,3,3.99,5,6.99,7,9.99,10,11.99,12,14.99,15,19.99,20,24.99])
forw_flat = np.array([0.042,0.042,0.04400096,0.04400096,0.05504608,0.05504608,0.064587848,0.064587848,0.064537411,0.064537411,0.067033154,0.067033154,0.066014169,0.066014169,0.051037629,0.051037629,0.047050895,0.047050895,0.053502366,0.053502366])

delta = 0.01

def piecewise_linear(T, swap_rates):
    S = interp1d(maturities, 
                 swap_rates, 
                 kind='linear', 
                 fill_value='extrapolate')
    return S(T)

def disc_factor(T, swap_rates):
    return ((1 + piecewise_linear(T, swap_rates)) ** (-T))

def forw_rate_piecewise_linear(T, swap_rates):
    return ((disc_factor(T, swap_rates) / disc_factor(T+delta, swap_rates)) ** (1 / delta) - 1)

ts = pd.DataFrame(np.arange(0,25 + delta, delta))
curve = ts.copy()
forw_1 = ts.copy()

for i in range(len(ts)):
    T = ts.iloc[i]
    curve.iloc[i] = piecewise_linear(T, swap_rates)
    forw_1.iloc[i] = forw_rate_piecewise_linear(T,swap_rates)

samlet = pd.concat([ts, curve], axis = 1)
samlet_2 = pd.concat([ts,forw_1], axis = 1)

fig, ax = plt.subplots(1,1, figsize=(w,h))

ax.set_xlim(0, 25)
ax.set_ylim(0.03,0.08)
ax.set_yticks(np.arange(0.03, 0.09, 0.01))
ax.grid(ls = (0,(1,2)))
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1, decimals = 0))
plt.plot(samlet.iloc[:,0], samlet.iloc[:,1], label = 'Piecewise Linear Yield Curve', color ='r')
plt.plot(maturities_2, forw_flat, label = 'Piecewise Flat Forward Curve', color='g')
plt.plot(samlet_2.iloc[:,0], samlet_2.iloc[:,1], label = 'Piecewise Linear Forward Curve', color='b')
plt.xlabel('T (Years)')
plt.legend(bbox_to_anchor=(0.55, 1.015),frameon=False,fontsize=9)
fig.tight_layout()
plt.savefig('C:/Users/Mikkel/OneDrive/CBS/CBS - FIN/THESIS/Main/python/billeder/bootstrapped_curves.pgf')
plt.show()

C:\Users\Mikkel\AppData\Local\Temp\ipykernel_7608\1627372894.py:48: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [8]:
# cubic spline
maturities = np.array([0, 1,2,3,5,7,10,12,15,20,25])
swap_rates = np.array([0.042, 0.042, 0.043, 0.047, 0.054, 0.057, 0.06, 0.061, 0.059, 0.056, 0.0555])

delta = 0.01

def cubspline_curve(T, swap_rates):
    S = interp1d(maturities, 
                 swap_rates, 
                 kind='cubic', 
                 fill_value='extrapolate')
    return S(T)

def disc_factor(T, swap_rates):
    return (np.exp(- cubspline_curve(T, swap_rates) * T))

def forw_rate(T, swap_rates):
    return (disc_factor(T, swap_rates) / disc_factor(T+delta, swap_rates) - 1) / delta

ts = pd.DataFrame(np.arange(0,25 + delta, delta))
curve = ts.copy()
forw = ts.copy()

for i in range(len(ts)):
    date = datetime.date(2023,1,4)
    T = ts.iloc[i]
    curve.iloc[i] = cubspline_curve(T, swap_rates)
    forw.iloc[i] = forw_rate(T, swap_rates)

samlet = pd.concat([ts, curve], axis = 1)
samlet_2 = pd.concat([ts,forw], axis = 1)

fig, ax = plt.subplots(1,1, figsize=(w,h))

ax.set_xlim(0, 25)
ax.set_ylim(0.03,0.08)
ax.set_yticks(np.arange(0.03, 0.09, 0.01))
ax.grid(ls = (0,(1,2)))
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1, decimals = 0))
plt.plot(samlet.iloc[:,0], samlet.iloc[:,1], label = 'Yield Curve', color = 'r')
plt.plot(samlet_2.iloc[:,0], samlet_2.iloc[:,1], label = 'Forward Curve', color = 'b')
plt.xlabel('T (Years)')
plt.legend(bbox_to_anchor=(1.005, 0.99),frameon=False,fontsize=9)
fig.tight_layout()
plt.savefig('C:/Users/Mikkel/OneDrive/CBS/CBS - FIN/THESIS/Main/python/billeder/cubic_spline_curves.pgf')
plt.show()

C:\Users\Mikkel\AppData\Local\Temp\ipykernel_10160\90518043.py:46: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## curve on sep 18, 2023

In [3]:
import numpy as np
import os
from scipy.stats import norm
import scipy.optimize as opt
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})
from scipy.interpolate import interp1d
from pandas.tseries.offsets import DateOffset
from dateutil.relativedelta import relativedelta
from scipy.optimize import fsolve, root
import datetime
import matplotlib.ticker as mtick 
import datetime
import statsmodels.api as sm

In [4]:
# latex_width = 6.69423 inches
w = 6.69423 * 9/10
golden_ratio = (5**.5 - 1) / 2
h = w * golden_ratio
h

3.7235355016582883

In [5]:
# curve data
os.chdir('C:/Users/Mikkel/OneDrive/CBS/CBS - FIN/THESIS/Main/data 13 11/')
EUSA = pd.read_excel('EUSA_data.xlsx')

# change dataframe layout
index = EUSA['Dates'].copy()
EUSA = pd.melt(EUSA.reset_index(),
               id_vars=['Dates'],
               value_vars=['6M', '1Y', '2Y', '3Y', '4Y', '5Y', '6Y', '7Y', '8Y', '9Y', '10Y', '12Y', '15Y', '20Y', '25Y', '30Y'],
               var_name='Maturity', value_name='Spot rate').set_index(['Dates'])
EUSA.name = 'EUSA'

# convert strings to float
def maturity_convert(Maturity):
    if Maturity[-1] == 'M':
        MaturityInYear = float(Maturity[:-1]) / 12  # if 6M -> 6/12
    else:
        MaturityInYear = float(Maturity[:-1])       # if xY -> float(x)
    return MaturityInYear

EUSA['Maturity (years)'] = EUSA['Maturity'].apply(lambda x: maturity_convert(x))

# change % to decimal
EUSA['Spot rate (decimal)'] = EUSA['Spot rate'].apply(lambda x: x / 100)

# cubic spline interpolation
def cubspline_curve(date, T, df_curve):
    S = interp1d(df_curve['Maturity (years)'][str(date)], 
                 df_curve['Spot rate (decimal)'][str(date)], 
                 kind='cubic', 
                 fill_value='extrapolate')
    return S(T)

def create_cubic_curve_cc(t, T, tau, date, curve_data): # returns cubic spline interpolated curve in cont. comp. rates
    ts = pd.Series(np.arange(t, T+tau, tau))
    curve = ts.copy()

    for i in range(len(ts)):
        tt = ts.iloc[i]
        curve.iloc[i] = np.log(1 + cubspline_curve(date, tt, curve_data))

    samlet = pd.DataFrame(curve, columns =['Rate']).set_index(ts.values)
    samlet.index.name = 'T'

    return samlet

date_19 = datetime.date(2019,9,18)
cubic_swap_curve_19 = create_cubic_curve_cc(0, 30, 0.25, date_19, EUSA)

date_20 = datetime.date(2020,9,18)
cubic_swap_curve_20 = create_cubic_curve_cc(0, 30, 0.25, date_20, EUSA)

date_21 = datetime.date(2021,9,20)
cubic_swap_curve_21 = create_cubic_curve_cc(0, 30, 0.25, date_21, EUSA)

date_22 = datetime.date(2022,9,19)
cubic_swap_curve_22 = create_cubic_curve_cc(0, 30, 0.25, date_22, EUSA)

date_23 = datetime.date(2023,9,18)
cubic_swap_curve_23 = create_cubic_curve_cc(0, 30, 0.25, date_23, EUSA)


fig, ax = plt.subplots(1,1, figsize=(w,h))

ax.set_xlim(0, 30)
ax.set_ylim(-0.01, 0.05)
ax.set_yticks(np.arange(-0.01, 0.06, 0.01))
ax.grid(ls = (0,(1,2)))
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1, decimals = 0))


plt.plot(cubic_swap_curve_23.index, cubic_swap_curve_23, color = 'b', label = 'Sep, 2023')
plt.plot(cubic_swap_curve_22.index, cubic_swap_curve_22, color = 'r', label = 'Sep, 2022')
plt.plot(cubic_swap_curve_21.index, cubic_swap_curve_21, color = 'g', label = 'Sep, 2021')
plt.plot(cubic_swap_curve_20.index, cubic_swap_curve_20, color = '#ff7f0e', label = 'Sep, 2020')
plt.plot(cubic_swap_curve_19.index, cubic_swap_curve_19, color = 'm', label = 'Sep, 2019')

plt.legend(frameon=False, bbox_to_anchor = (1.005, 1.015), fontsize=8)
plt.xlabel('T (Years)')
fig.tight_layout()
plt.savefig('C:/Users/Mikkel/OneDrive/CBS/CBS - FIN/THESIS/Main/python/billeder/euribor_curve_18sep23.pgf')